In [1]:
%matplotlib inline


神经网络
===============

神经网络可以通过``torch.nn``包来创建。

我们之前简单的了解了``autograd``，而``nn``会使用``autograd``来定义模型以及求梯度。
一个``nn.Module``对象包括了许多网络层(layer)，并且有一个``forward(input)``方法来返回``output``。

比如下图所示，我们会定义一个卷积网络来识别mnist图片

![识别mnist的网络](mnist.png "识别mnist的网络")

训练一个神经网络通常需要如下步骤：

- 定义一个神经网络，它通常有一些可以训练的参数
- 迭代一个数据集(dataset)
- 处理给网络的输入
- 计算loss(会调用Module对象的forward方法)
- 计算loss对参数的梯度
- 更新参数，通常使用如下的梯度下降方法来更新：
  ``weight = weight - learning_rate * gradient``

调用神经网络
------------------
让我们首先来定义网络：


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 输入是1个通道的灰度图，输出6个通道(feature map)，使用5x5的卷积核
        self.conv1 = nn.Conv2d(1, 6, 5)
        # 第二个卷积层也是5x5，有16个通道
        self.conv2 = nn.Conv2d(6, 16, 5)
        # 全连接层
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # 32x32 -> 28x28 -> 14x14 
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # 14x14 -> 10x10 -> 5x5
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # 除了batch维度之外的其它维度。
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


你只需要定义``forward``函数，而``backward``函数会自动通过``autograd``创建。
在``forward``函数里我们可以使用任何处理Tensor的函数。

可以使用函数``net.parameters()``来得到模型所有的参数。



In [3]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1的weight

10
torch.Size([6, 1, 5, 5])


尝试一个随机的32x32的输入
注意：这个网络(LeNet)期望的输入大小是32x32。如果使用MNIST数据集(28x28)，我们需要缩放到32x32。



In [4]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.1198,  0.0230,  0.0322, -0.0951,  0.1117,  0.0377, -0.0365, -0.1305,
          0.1472,  0.0533]], grad_fn=<ThAddmmBackward>)


默认的梯度会累加，因此我们通常在backward之前清除掉之前的梯度值：



In [5]:
net.zero_grad()
out.backward(torch.randn(1, 10))

<div class="alert alert-info"><h4>注意</h4><p>``torch.nn``只支持mini-batches的输入。整个``torch.nn``
    包的输入都必须第一维是batch，即使只有一个样本也要弄成batch是1的输入。
    

    <p>比如，``nn.Conv2d``的输入是一个4D的Tensor，shape是
    ``nSamples x nChannels x Height x Width``。

    如果你只有一个样本(nChannels x Height x Width)，那么可以使用``input.unsqueeze(0)``来增加一个batch维。</p></div>

在继续学习之前，我们先简单的复习一下已经学过的一些类。


**复习：**
  -  ``torch.Tensor`` - 一个*多维数组*，在(依赖它的tensor)调用``backward()``时支持自动求导。它也会把计算得到的梯度保存起来。
  -  ``nn.Module`` - 神经网络模块。可以方便的封装参数。
  -  ``nn.Parameter`` - 一种特殊的Tensor，当我们把它作为Module的属性的时候，它会自动注册为这个模块的参数。、
  -  ``autograd.Function`` - 实现了一个autograd operation的前向和后向定义。 每个``Tensor``的operation，都至少会创建一个``Function``节点， 它会和``Tensor``简历关联并且记录相关用于自动梯度计算需要的历史信息。

**到目前位置，我们介绍了：**
  -  定义神经网络
  -  处理输入和进行前向计算，调用backward计算梯度。
  
**尚未介绍内容：**
  -  计算loss
  -  更新网络参数
  
损失函数
-------------
损失函数的参数是(output, target)对，output是模型的预测，target是实际的值。损失函数会计算预测值和真实值的差别，损失越小说明预测的越准。

这里有许多不同的损失函数：
`loss functions <http://pytorch.org/docs/nn.html#loss-functions>`

最简单的一个损失函数是：``nn.MSELoss``，它会计算预测值和真实值的均方误差。

比如：


In [6]:
output = net(input)
target = torch.randn(10)  # 随便伪造的一个“真实值” 
target = target.view(1, -1)  # 把它变成output的shape(1, 10) 
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(1.3318, grad_fn=<MseLossBackward>)


如果从``loss``往回走，需要使用tensor的``.grad_fn``属性，我们Negative看到这样的计算图：


::

    input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
          -> view -> linear -> relu -> linear -> relu -> linear
          -> MSELoss
          -> loss

因此当调用``loss.backward()``时，PyTorch会计算这个图中所有``requires_grad=True``的tensor关于loss的梯度。



In [7]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Add
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # Expand

反向梯度计算
--------
在调用``loss.backward()``之前，我们需要清除掉tensor里之前的梯度，否则会累加进去。




In [8]:
net.zero_grad()     # 清掉tensor里缓存的梯度值。

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0035, -0.0056, -0.0081,  0.0094, -0.0202,  0.0192])


接下来我们来看怎么使用损失函数。

**深入阅读：**

  torch.nn包括了很多模块和损失函数，这是深度神经网络的基础。完整的文档在`这里 <http://pytorch.org/docs/nn>`

**最后需要介绍的内容：**

  - 更新神经网络的参数。
  
更新参数
------------------
更新参数最简单的方法是使用随机梯度下降(SGD)：

     ``weight = weight - learning_rate * gradient``

我们可以使用如下简单的代码来实现更新：

.. code:: python

    learning_rate = 0.01
    for f in net.parameters():
        f.data.sub_(f.grad.data * learning_rate)

通常我们会使用更加复杂的更新方法，比如SGD, Nesterov-SGD, Adam, RMSProp等等。
为了实现这些算法，我们可以使用``torch.optim``包，它的用法也非常简单：




In [9]:
import torch.optim as optim

# 创建optimizer，需要传入参数和learning rate
optimizer = optim.SGD(net.parameters(), lr=0.01)

# 清除梯度
optimizer.zero_grad()  
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # optimizer会自动帮我们更新参数

.. 注意::

      即使使用optimizer，我们也需要清零梯度。我们不需要一个个的清除，只需要用
      ``optimizer.zero_grad()``。

